# Amazon Dataset

Datasets from: https://nijianmo.github.io/amazon/index.html#complete-data  
Take ratings only data (I downloaded ratings-only, magazine subscription as an example.)

In [23]:
!pip install hdf5storage

     |████████████████████████████████| 61kB 4.0MB/s 


In [24]:
import pandas as pd
import numpy as np
import scipy.io as sio
import h5py
import hdf5storage

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/AMAZON_FASHION_5.json.gz

--2021-03-21 04:36:34--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/AMAZON_FASHION_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287013 (280K) [application/octet-stream]
Saving to: ‘AMAZON_FASHION_5.json.gz’

AMAZON_FASHION_5.js 100%[===================>] 280.29K   468KB/s    in 0.6s    

2021-03-21 04:36:35 (468 KB/s) - ‘AMAZON_FASHION_5.json.gz’ saved [287013/287013]



In [4]:
import json
import pandas as pd
import gzip


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/AMAZON_FASHION_5.json.gz')

In [5]:
new_df = df.drop(labels = ['style', 'verified', 'reviewTime', 'reviewerName',
                  'reviewText', 'summary', 'vote', 'image'], axis=1)
new_df = new_df.rename(columns = {'overall': 'rating', 'reviewerID' : 'user', 'asin': 'item', 'unixReviewTime' : 'timestamp'})

In [6]:
data = new_df

In [ ]:
# Mount drive with button (on the left vertical bar, under files)
# Copy file path

data = pd.read_csv('/content/drive/MyDrive/CS6101/Guan Hao/Sample Datasets/Magazine_Subscriptions.csv', names = ['item', 'user', 'rating', 'timestamp'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data.head()

,rating,user,item,timestamp
0,5.0,ALJ66O1Y6SLHA,B000K2PJ4K,1441324800
1,5.0,ALJ66O1Y6SLHA,B000K2PJ4K,1441324800
2,5.0,ALJ66O1Y6SLHA,B000K2PJ4K,1441324800
3,5.0,ALJ66O1Y6SLHA,B000K2PJ4K,1441324800
4,5.0,ALJ66O1Y6SLHA,B000K2PJ4K,1441324800


In [ ]:
data.shape

(1143860, 4)

In [8]:
data = data.drop(['timestamp'], axis = 1).groupby(['item', 'user']).aggregate(np.mean).reset_index()

In [9]:
data['rating'] = [int(np.round(x)) for x in data['rating']]

In [10]:
set(data['rating'])

{1, 2, 3, 4, 5}

In [11]:
data.head()

,item,user,rating
0,B000K2PJ4K,A3W11493KS6Z2L,4
1,B000K2PJ4K,ALJ66O1Y6SLHA,5
2,B000KPIHQ4,A265UZVOZWTTXQ,5
3,B000KPIHQ4,A3HX4X3TIABWOV,2
4,B000KPIHQ4,AW8UBYMNJ894V,4


In [12]:
data.shape

(3042, 3)

In [13]:
for category in ['item', 'user']:
  lst = list(set(data[category]))
  lst.sort()
  lst = [tup[::-1] for tup in list(enumerate(lst))]
  curr_dict = dict(lst)
  data[category] = [curr_dict[id] for id in data[category].values]

In [14]:
len(set(data['user']))

406

In [15]:
88318 / (72098 * 2428)

0.0005045187670835911

In [16]:
len(set(data['item']))

31

In [17]:
matrix_df = data.pivot_table(index = 'user', columns = 'item', values = 'rating').fillna(0).astype(int)

In [ ]:
matrix_df.head()

item,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,11757,11758,11759,11760,11761,11762,11763,11764,11765,11766,11767,11768,11769,11770,11771,11772,11773,11774,11775,11776,11777,11778,11779,11780,11781,11782,11783,11784,11785,11786,11787,11788,11789,11790,11791,11792,11793,11794,11795,11796
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
matrix_df.apply(lambda x: x > 0).sum(axis = 1).describe()

count    16566.000000
mean         8.770494
std         10.212211
min          1.000000
25%          5.000000
50%          6.000000
75%          9.000000
max        557.000000
dtype: float64

In [ ]:
matrix_df.apply(lambda x: x > 0).sum(axis = 0).describe()

count    11797.000000
mean        12.316013
std         15.689818
min          1.000000
25%          6.000000
50%          8.000000
75%         13.000000
max        572.000000
dtype: float64

In [ ]:
matrix_df.sum(axis = 0).describe()

count    11797.000000
mean        57.849368
std         73.770665
min          2.000000
25%         25.000000
50%         37.000000
75%         63.000000
max       2762.000000
dtype: float64

In [ ]:
# set(np.ndarray.flatten(matrix_df.values))

In [18]:
matrix_df.shape

(406, 31)

# Flixster Example Mat File

In [ ]:
# Reference: https://stackoverflow.com/questions/17316880/reading-v-7-3-mat-file-in-python
# No idea what the W matrices represent

dataset_name = 'douban' # (can switch to 'yahoo_music' and 'flixster')
filepath = f'/content/drive/MyDrive/CS6101/Guan Hao/Sample Datasets/training_test_dataset_{dataset_name}.mat'
example_data = {}
f = h5py.File(filepath)
for k, v in f.items():
    example_data[k] = np.array(v)
    print(k)
    print(example_data[k].shape)
    print(np.unique(example_data[k]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  if __name__ == '__main__':


M
(3000, 3000)
[0. 1. 2. 3. 4. 5.]
Otest
(3000, 3000)
[0. 1.]
Otraining
(3000, 3000)
[0. 1.]
W_users
(3000, 3000)
[0. 1.]


In [ ]:
# Checks (Should have no output)

for i in range(3000):
  for j in range(3000):
    if example_data['Otest'][i][j] == 1 or example_data['Otraining'][i][j] == 1:
      if example_data['M'][i][j] == 0:
        print((i, j))

    if example_data['Otest'][i][j] == 0 and example_data['Otraining'][i][j] == 0:
      if example_data['M'][i][j] > 0:
        print((i, j))

NameError: ignored

In [ ]:
# Uncomment to see examples

# example_data['M']
# example_data['Otraining']
# example_data['Otest']
# example_data['W_users']

# Train Test Split for Amazon Dataset

In [19]:
np.random.seed(1)

threshold = 20 # testing split percentage (example)

# train_test_split
matrix = matrix_df.values
Otraining_matrix = matrix.copy()
Otest_matrix = matrix.copy()

for i in range(matrix.shape[0]):
  for j in range(matrix.shape[1]):
    if matrix[i][j] > 0:
      random_int = np.random.randint(0, 100)

      if random_int < threshold:
        Otest_matrix[i][j] = 1
        Otraining_matrix[i][j] = 0

      else:
        Otraining_matrix[i][j] = 1
        Otest_matrix[i][j] = 0

In [20]:
matrix.shape

(406, 31)

In [25]:
# Links: 
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.savemat.html
# https://stackoverflow.com/questions/35706697/saving-numpy-structure-array-to-mat-file

matfiledata = {} # make a dictionary to store the MAT data in
matfiledata[u'variable1'] = np.zeros(100) # *** u prefix for variable name = unicode format, no issues thru Python 3.5; advise keeping u prefix indicator format based on feedback despite docs ***
matfiledata[u'variable2'] = np.ones(300)

mdic = {u"M": matrix, u"Otraining": Otraining_matrix, u"Otest": Otest_matrix}
# sio.savemat('/content/amazon_vg.mat', mdic)
hdf5storage.write(mdic, '.', 'train_test_dataset.mat', matlab_compatible=True)

In [ ]:
if x:
    print('Hi')
else:
    print()

NameError: ignored